# Causation (Chain of Thoughts Self Consistency)

Chain of Thoughts is a prompting technique that use examples containing steps to assist with the reasoning ability of a large language model (llm).

Self consistency is a layer that adds on top that leverage the probabilistic outputs from LLMs and take the majority vote as the final answer.

This notebook is an implementation of this for the GEF causation project.

## OpenAI Privacy Policy
This notebook uses OpenAI's API, meaning that your data will be sent to the OpenAI servers.

For concerns about how your data will be handled, please read through the Privacy Policy [here](https://openai.com/policies/api-data-usage-policies).

## Paper: Chain of Thoughts (expand to read)
Chain of Thought (CoT): https://arxiv.org/abs/2201.11903<br>
CoT Self Consistency (CoT-SC): https://arxiv.org/abs/2203.11171

In [ ]:
from IPython.display import IFrame
display(IFrame(src='https://arxiv.org/pdf/2201.11903.pdf', width=1600, height=700))

In [ ]:
from IPython.display import IFrame
# CoT - Self Consistency
display(IFrame(src='https://arxiv.org/pdf/2203.11171.pdf', width=1600, height=700))

## Overview
1. Supply your OpenAI API Key
2. Upload your file with the Chain of Thoughts examples.
3. Choose a sampling scheme and number of completions
4. Enter a query (i.e. the sentence) and run it for a classification.
5. Batch classification using your dataset. **requires: sentence, det, se, hom, nat columns**

## 1. Enter your OpenAI API Key.

To create an API Key, please go to: https://platform.openai.com/account/api-keys

In [ ]:
from causation.utils import openai_apikey_input

openai_apikey_input()

## 2. Upload your Chain of Thoughts examples

Chain of thoughts uses examples to help the LLM reason about your queries better by breaking it down into steps.

The idea is to give it more context and prompt it to break down its reasoning process into steps.

👼 Experiment with different 'steps' for each example, I would recommend having a diverse set of reasons and be specific. <br> 
👼 It's usually better with more examples but this also means it'll increase your API cost (see TikDollar later). Starting out with 3-5 examples per class should be sufficient.

In [ ]:
from causation.utils import fileuploader 

finput, exemplars = fileuploader('.toml')
finput

In [ ]:
# encoding issue - 27.Sep.23 - this strips away the whitespace/end-of-line character at the end of each line.
# it's okay to always run this cell with or without the encoding issue.
!sed 's/" $/"/g' "{exemplars.get('data').absolute()}" > "{exemplars.get('data').absolute()}.formatted"
!mv "{exemplars.get('data').absolute()}.formatted" "{exemplars.get('data').absolute()}"

In [ ]:
assert exemplars.get('data'), "Did you upload your CoT examples in the previous cell?"

from llm_experiments import CoT

cot = CoT.from_toml(exemplars.get('data'))
cot.shuffle_examples()                                         # improves result
f"These CoT example class distributions: {cot.class_dist()}"   # try to keep this balanced.

👼 Currently supported models: text-davinci-003, text-davinci-002, text-curie-001, text-babbage-001, text-ada-001
(In decreasing performance and cost.)<br>
👼 Newly Added: gpt-3.5-turbo at 10% of the cost of text-davinici-003 but similar performance!

Note: If you enter the wrong model name, it'll tell you what's available in the error message.

### [Optional] Sample only a few examples.

In [ ]:
# [Optional] - randomly sample X CoT examples to reduce input tokens.
cot.sample(method='random', n=len(cot.examples))
cot.class_dist()

## 3. Choose a sampling scheme and the number of completions.

__Sampling Scheme__:
+ `top_p` - **the range of output words within a probability threshold.** (0 < top_p <= 1.0)<br>
e.g. <br>
= 0.3 means sample only from outputs that make up the top 30% of the probabilities.

+ `temperature` - **the higher the temperature the more spread out the probabilities are across the output words.** (0 <= temperature <= 2) <br>
e.g. <br>
= 0 means output probabilities are *not* spread out i.e. Only 1 output token have a probability of 1.0 when the LLM is generating the output. This means it will always choose the same output during the decoding ending up with the same completion.<br>
\> 0, ~0 means output probabilities are a little spread out i.e. > 1 output tokens will have a probability of closer to 1.0, combining to 1.0. This means that the LLM will have a chance of sampling different output tokens.<br>
= 2 means output probabilities are *most* spread out i.e. output tokens will have similar probabilities and the LLM will have similar chance of sampling from each token.

`top_p` and `temperature` goes hand in hand. Having a really low temperature means there are less output tokens within the probability threshold.

__Number of completions__ (relates to: self-consistency):
+ `n_completions` - **this is the number of responses you're asking the LLM to generate.**<br>(increases cost but directly relates to the number of votes used by Self Consistency)

__Penalties__:
+ `presence_penalty` - **this forces the model to be more creative in their word choices per completion. i.e. it penalises words that the model have already said.** (-2.0 <= presence_penalty <= 2.0)<br>
empirically, this have shown to output more tokens if increased.

<br>
👼 Experiment with different sampling schemes and increase number of completions for more confidence in your votes (beware of costs!)

In [ ]:
from llm_experiments import SamplingScheme

sampling_scheme = SamplingScheme(top_p=0.8, temperature=1, presence_penalty=0.0)
n_completions = 3

assert n_completions > 1, "For the model to generate > 1 possibilities needed for self-consistency, n_completions must be > 1."
assert sampling_scheme.temperature > 0, "For the model to generate > 1 possibilities needed for self-consistency, temperature must be > 0."
sampling_scheme

In [ ]:
from llm_experiments import CoTSC

cotsc = CoTSC.from_cot(model='gpt-3.5-turbo',  # for a larger context window (from 4k -> 16k tokens) replace with 'gpt-3.5-turbo-16k'
                       cot=cot,
                       sampling_scheme=sampling_scheme,
                       n_completions=n_completions)
f"{cotsc.model}   'temperature': {cotsc.llm.temperature}, {str(cotsc.llm.model_kwargs).lstrip('{').rstrip('}')}"

In [ ]:
prompt, num_tokens = cotsc.dryrun(query="Canberra immunologist Carola Vinuesa who discovered a gene responsible for the autoimmune diseases lupus and diabetes.")
print(prompt)
f"Number of tokens in the above prompt: {num_tokens}"

### Here's an example (Sent to OpenAI)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 250)

results = cotsc.run(query="Canberra immunologist Carola Vinuesa who discovered a gene responsible for the autoimmune diseases lupus and diabetes.")
pd.DataFrame.from_dict(results, orient='index')

### TikDollar
**Tikdollar is created by SIH to track your OpenAI expense and cut if off at a specified threshold.<br>
The code will run your calls right up until when your next call will exceed this threshold.**

> OpenAI charges for both your input and output tokens. Each token can be thought of as a word in the normal sense but tokens for LLMs are actually *subwords*.

For more information on subwords here's a cool video: https://www.youtube.com/watch?v=zHvTiHr506c

**Now, we're going to bind TikDollar with our CoTSC calling function.**

The parameters you need to care about:
+ `cost_threshold`  - this is the cut-off USD. You'll need to define this. (e.g. 0.1, 1.0, 20)
+ `raise_err` - when the cut-off will be exceeded in the next call, stop. (or print a message then continue if it's False)
+ `verbose` - whether to print messages per call in terms of your spending.

In [ ]:
from llm_experiments.utils import TikDollar as td

# ⚠️ Caveat: When you rerun this cell, tikdollar is reset to 0!
tikdollar = td.track(cotsc, cotsc._tikdollar_run, cost_threshold=0.1, raise_err=True, verbose=True)
tikdollar  # starts out with zero cost accumulated.

In [ ]:
_ = cotsc.run(query="Professor Lawford says the institute, based at the Greenslopes Private Hospital, in Brisbane's south, in conjunction with overseas collaborators, is analysing the DNA of Australian Vietnam veterans in a bid to better understand the causes of PTSD.")

In [ ]:
tikdollar # cost after you call the API.

## 4. Batch Classification

Now that you have **CoTSC** and **TikDollar** you're equipped to run a classification task on your list of sentences!

At the end of this there'll be a link for you to click on to download all the generated results into an excel sheet.

#### 1. Upload your dataset (requires: 'sentence' column)

In [ ]:
from causation.utils import fileuploader

finput, dataset = fileuploader('.xlsx')
finput

In [ ]:
import pandas as pd
assert dataset.get('data'), "Did you upload your dataset?"
df = pd.read_excel(dataset.get('data'))
df.head(1)

In [ ]:
# validate uploaded dataset
required_columns = ['sentence', 'det', 'se', 'hom', 'nat']
for req_col in required_columns:
    assert req_col in df.columns, f"Missing column: '{req_col}' in {list(df.columns)}"
f"All required columns found: {required_columns}. Please continue."

In [ ]:
if 'pos' in df.columns:
    import sys
    print("- [warn] Found 'pos' column in your dataset, "
          "these values will be recomputed to represent whether any bias exist. "
          "Do not worry about this warning message. "
          "(This will not overwrite your existing excel file.)", 
          file=sys.stderr)
df['pos'] = df.loc[:, ['det', 'se', 'hom', 'nat']].sum(axis=1).apply(lambda sum_: min(sum_, 1))
assert 'pos' in df.columns, "Missing 'pos' column. This should not happen after it's been added"
f"'pos' column now represents whether any 'bias' exist. Please continue."

In [ ]:
# Check and fill NaN values with 0 for label columns
label_columns = ['det', 'se', 'hom', 'nat']
hasnans = df.loc[:, label_columns].isnull().values.any()
if hasnans:
    import sys
    print("- [warn] Found NaN values in the 'det', 'se','hom', 'nat' columns. "
          "These values will be replaced with 0s.",
          file=sys.stderr)
    for col in label_columns:
        df[col] = df[col].fillna(0)

hasnans = df.loc[:, label_columns].isnull().values.any()
assert not hasnans, f"There are still NaN values in the label columns: {label_columns}. This should not happen."
f"Label columns {label_columns} checked to not have any NaN values. Please continue."

In [ ]:
# plot class distribution
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

fig = make_subplots(rows=1, cols=4)

fig.add_trace(go.Histogram(x=df['det']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['hom']), row=1, col=2)
fig.add_trace(go.Histogram(x=df['se']), row=1, col=3)
fig.add_trace(go.Histogram(x=df['nat']), row=1, col=4)

fig.update_xaxes(title_text="determinism", dtick=1, row=1, col=1)
fig.update_xaxes(title_text="homogeneity", dtick=1, row=1, col=2)
fig.update_xaxes(title_text="specific aetiology", dtick=1, row=1, col=3)
fig.update_xaxes(title_text="naturalism", dtick=1, row=1, col=4)

fig.update_yaxes(title_text="Count", range=[0, len(df)])

# Update layout
fig.update_layout(height=300, width=1400, title_text="Distribution of Classes", showlegend=False)

fig.show()

#### 2. Set up TikDollar

In [ ]:
# Setup TikDollar
# copy of prior cell for task separation and easy access.
from llm_experiments.utils import TikDollar as td

# ⚠️ Caveat: When you rerun this cell, tikdollar is reset to 0!
tikdollar = td.track(cotsc, cotsc._tikdollar_run, cost_threshold=20, raise_err=True, verbose=False)
tikdollar

#### 3. Run classification on uploaded dataset

🧑‍💻 If you find the TikDollar messages cluttering your screen, set `verbose=False` in the previous cell, run the cell and then run the cell below.

**Data leakage**:<br>
We expect the examples in your dataset to not be one of your CoT examples. When it is, the data has leaked as the model will just answer what you've provided leading to a biased evaluation. These examples are skipped.

In [ ]:
from llm_experiments.utils.tikdollar import CostThresholdReachedException
from llm_experiments.cot.cot import CoTDataLeakException
from collections import namedtuple
from tqdm.auto import tqdm

ROW = namedtuple('ROW', ['query', 'clazz', 'votes', 'steps', 'determinism', 'specific_aetiology', 'naturalness', 'homogeneity', 'is_biased', 'completions'])

Rows = list()
dleak_counter = 0
for i, row in tqdm(enumerate(df.itertuples()), total=len(df)):
    query = row.sentence
    det, se, nat, hom, pos = row.det, row.se, row.nat, row.hom, row.pos
    try:
        results = cotsc.run(query=query)
        for clazz, clz_res in results.items():
            Row = ROW(query=query, clazz=clazz, votes=clz_res.get('votes'), steps=clz_res.get('steps'), 
                      determinism=det, specific_aetiology=se, naturalness=nat, homogeneity=hom, is_biased=pos, completions=clz_res.get('completions'))
            Rows.append(Row)
    except CostThresholdReachedException as ctre:
        print(ctre)
        print(f"Number of queries sent: {i}.")
        break
    except CoTDataLeakException as cotdle:
        print(cotdle)
        print("Data leak detected. Skipped.")
        dleak_counter += 1
        continue

print(f"Number of examples leaked: {dleak_counter}")
results_df = pd.DataFrame(Rows)

Data leakage is when a 'query' is matched exactly with one of the examples given in your CoT examples (i.e. the training examples in the .toml file you've uploaded earlier). These queries are not sent to OpenAI as the model will give the exact answer you've given in your example which is not representative of the model's accuracy, and I suppose it also doesn't require a classification.

In [ ]:
results_df = pd.DataFrame(Rows)
f"Obtained {len(results_df)} results."

The above number of results may not match the number of queries.
This is because of *self-consistency* (see `n_completions` argument in section 3) and model may generate multiple answers for one query. The majority vote is taken as the classification.

#### 4. Analyse classification results

In [ ]:
# Let's take a quick look at the results
results_df.head(1)

In [ ]:
results_df.clazz.value_counts()

In [ ]:
import sys
import numpy as np

clazz_to_idx = {
    "determinism": 0,
    "specific_aetiology": 1,
    "naturalness": 2,
    "homogeneity": 3,
}

def clazz_to_onehot(clazz: str) -> list[int]:
    # clean up llm output
    try:
        comma_idx = clazz.index(',')
        clazz = clazz[:comma_idx]
    except:
        pass
    clazz = clazz.strip()
    row = [0, 0, 0, 0]
    if clazz in ('not_biased', 'neutral'): 
        return row
    elif clazz_to_idx.get(clazz, None) is None:
        print(f"{clazz} is not one of {', '.join(clazz_to_idx.keys())}. Continued as 'neutral'", file=sys.stderr)
        return row
    else:
        row[clazz_to_idx.get(clazz)] = 1
        return row

preds, targets = list(), list()
for query, group in results_df.groupby(by='query'):
    best_idx = group.loc[:, 'votes'].idxmax()   # ties are randomly selected here.
    best = group.loc[best_idx]
    prediction = clazz_to_onehot(best.clazz)
    target = group[['determinism', 'specific_aetiology', 'naturalness', 'homogeneity']].values[0]
    preds.append(prediction)
    targets.append(target)

preds, targets = np.array(preds), np.array(targets)
assert preds.shape == targets.shape, "Mismatched shape between prediction and targets. This should not happen."
"Computed values to generate classification report. Please continue."

In [ ]:
# classification report
from sklearn.metrics import classification_report
import sys

def report(clazzes, targets, preds, file=sys.stdout):
    for i in range(len(clazzes)):
        print(f"=== {clazzes[i]} ===", file=file)
        print(classification_report(y_true=targets[:, i], y_pred=preds[:, i], zero_division=0), file=file)
        print("\n", file=file)
    
clazzes = list(clazz_to_idx.keys())
report(clazzes, targets, preds)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def cm(clazzes, targets, preds):
    num_rows, num_cols = 2, 2
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_rows, 5*num_rows))
    for i in range(len(clazzes)):
        clazz = clazzes[i]
        labels = ['neutral', clazz]
        y_true, y_pred = list(map(lambda x: labels[int(x)], targets[:, i])), list(map(lambda x: labels[int(x)], preds[:, i]))
        disp = ConfusionMatrixDisplay(confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels), display_labels=labels)
        r, c = i//2, i%2
        ax = axes[r, c]
        ax.set_title(f"{clazz}")
        disp.plot(ax=axes[r, c])
    return fig
fig = cm(clazzes, targets, preds)

In [ ]:
# single-target-multi-class confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def to_labels(onehot):
    """ This converts the [det, se, hom, nat] to [neutral, det, se, hom, nat] """
    num_classes_plus_neutral = onehot.shape[1] + 1
    labels = np.zeros((onehot.shape[0], num_classes_plus_neutral))
    for idx in range(len(onehot)):
        if np.sum(onehot[idx]) == 0:
            continue
        else:
            clz_idx = np.argmax(onehot[idx])
            clz_idx = clz_idx + 1
            labels[idx][clz_idx] = 1
    return labels

# dynamic display_labels (dynamic to the samples e.g. SE only, or SE,DET only where target is a subset of all biases)
all_labels = ['neutral'] + list(map(lambda name: name[:3] if len(name.split("_")) == 1 else "_".join([chunk[:2] for chunk in name.split("_")]), clazzes))
y_true=np.argmax(to_labels(targets), axis=1)
y_pred=np.argmax(to_labels(preds), axis=1)

y_true = list(map(lambda i: all_labels[i], y_true))
y_pred = list(map(lambda i: all_labels[i], y_pred))

fig_st, axes = plt.subplots(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix(
    y_true=y_true,
    y_pred=y_pred,
    labels=all_labels,
), display_labels=all_labels)
disp.plot(ax=axes)

## 5. Download Results
1. Full COTSC outputs. (cotsc-outputs.xlsx)
2. Classification results. (cotsc-results.txt)
3. COTSC model configuration (cotsc-config.json)
4. COTSC prompt toml (your uploaded toml file)
5. Dataset used for classification.

👼 Please run through all the cells. All results are packaged into `cotsc-results-{timestamp}.zip` at the final cell and a download link will appear.

In [ ]:
# create the temporary results directory
import os
import shutil
from pathlib import Path
dir_: Path = Path('./.causation-cotsc-results')   # note: hidden folder.
if dir_.exists(): shutil.rmtree(dir_)
os.makedirs(dir_, exist_ok=True)
assert dir_.exists(), "Temporary directory did not get created."
f"Results will be saved to this temporary directory: {dir_}"

In [ ]:
# reformat df for readability. (Adds veracity column)

from collections import namedtuple

Row = namedtuple('Row', ['query', 'majority_pred', 'veracity', 'votes', 'steps_per_vote', 'minority_preds', 'det', 'se', 'nat', 'hom', 'neutral', 'raw_outputs'])

rows = list()
formatted_results_df = results_df.copy()
formatted_results_df['neutral'] = 1 - formatted_results_df['is_biased'] 

for gid, group in formatted_results_df.groupby(by='query'):
    _targets = list(filter(lambda x: x in clazzes + ['neutral'], group.columns[group.eq(1, axis=1).any()].tolist()))  # list of positive _targets (it is multi-target)
    minority_preds = dict()
    if len(group) > 1:
        sorted_ = group.sort_values(by='votes', ascending=False)
        best = sorted_.iloc[0]
        minority = sorted_.iloc[1:]
        for m in minority.itertuples():
            minority_preds[m.clazz] = {'votes': m.votes, 'steps_per_vote': m.steps}
    else:
        best = group.iloc[0]
    
    # veracity
    if len(_targets) > 1:
        pred = best.clazz.lower()
        if 'neutral' in _targets:
            veracity = '(multi-target) contradictory targets'
        elif pred in _targets:
            veracity = '(multi-target) partially correct'
        else:
            veracity = '(multi-target) incorrect'
    elif len(targets) == 0:
        veracity = "no target found"
    else:
        target = _targets[0].lower()
        pred = best.clazz.lower()
        if target in clazzes and pred == target:
            veracity = 'true positive'
        elif target == 'neutral' and pred == target:
            veracity = 'true negative'
        elif pred in clazzes and target == 'neutral':
            veracity = 'false positive'
        elif pred == 'neutral' and target in clazzes:
            veracity = 'false negative'
        elif pred != target and pred in clazzes and target in clazzes and target not in cotsc.classes:
            veracity = 'unsupported classification'
        elif pred != target and pred in clazzes and target in clazzes:  # when > 1 bias in cotsc
            veracity = 'incorrect bias'
        else:
            pred_chunks = set(p.strip() for p in  pred.split(","))
            if target in pred_chunks:
                veracity = "malformed but partially correct"
            else:
                veracity = "malformed and incorrect"
    
    row = Row(query=best.query, 
              majority_pred=best.clazz, 
              veracity=veracity,
              votes=best.votes,
              steps_per_vote='\n\n'.join(best.completions),
              minority_preds=minority_preds,
              det=best.determinism,
              se=best.specific_aetiology,
              nat=best.naturalness,
              hom=best.homogeneity,
              neutral= best.neutral,
              raw_outputs=best.completions,
             )
    rows.append(row)

formatted_results_df = pd.DataFrame(rows)
formatted_results_df.head(3)

In [ ]:
# File 1: classification outputs
path = dir_.joinpath('cotsc-outputs.xlsx')
formatted_results_df.to_excel(path, index=False)

assert path.exists(), f"Failed to save to {path}"
"Added 'cotsc-outputs.xslx' to zip. Please continue."

In [ ]:
# File 2: classification evaluation results
path = dir_.joinpath('cotsc-results.txt')
with open(path, 'w') as f:
    clazzes = list(clazz_to_idx.keys())
    num_clazzes = preds.shape[1]
    report(clazzes, targets, preds, file=f)
    
assert path.exists(), f"Failed to save to {path}"
"Added 'cotsc-results.txt' to zip. Please continue."

In [ ]:
# File 3: classification evaluation confusion matrix
path = dir_.joinpath('cotsc-confusion-matrix.png')
path_st = dir_.joinpath('cotsc-confusion-matrix-st.png')
assert fig, "Did you run the confusion matrix cell earlier?"
assert fig_st, "Did you run the confusion matrix (single target) cell earlier?"
_ = fig.savefig(path)
_ = fig_st.savefig(path_st)
"Added confusion matrix to zip. Please continue."

In [ ]:
# File 4: cotsc configuration
import srsly

path = dir_.joinpath('cotsc-config.json')
cotsc_config = {
    'sampling_scheme': sampling_scheme.openai(),
    'n_completions': cotsc.n_completions,
    'model': cotsc.model,
    'classes': cotsc.classes,
}
srsly.write_json(path, cotsc_config)
assert path.exists(), f"Failed to save to {path}"

"Added cotsc configurations cotsc-config.json to zip. Please continue."

In [ ]:
# List of names of files to be added to the zip
file_names = [exemplars['data'], dataset['data']]  # toml & dataset
file_names += list(dir_.glob('*'))

print("These are the files that will be saved in the zip: ")
[f.name for f in file_names]

In [ ]:
# Open a zip file in write mode
# zip results and causation config
import zipfile
import os
from datetime import datetime
from pathlib import Path
import panel as pn

now = datetime.now().strftime(format="%Y-%m-%d_%H-%M-%S")
zfname = Path(f'cotsc-results_{now}.zip')
with zipfile.ZipFile(zfname, 'w') as zipf:
    for file_name in file_names:
        zipf.write(file_name, arcname=os.path.basename(file_name))
print(f"Saved as {zfname}.\nClick below to download.")

# download link for the zip.
pn.widgets.FileDownload(file=str(zfname), filename=zfname.name)